In [ ]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from datasets import load_dataset
from transformers import RobertaForMaskedLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from transformers import RobertaTokenizerFast
from transformers import BertForMaskedLM, BertTokenizer
import torch

In [ ]:
from transformers import RobertaTokenizerFast
import pandas as pd
from datasets import Dataset, DatasetDict
splits = {'train': 'plain_text/train-00000-of-00001.parquet', 'test': 'plain_text/test-00000-of-00001.parquet', 'unsupervised': 'plain_text/unsupervised-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/stanfordnlp/imdb/" + splits["train"])
dataset = Dataset.from_pandas(df)
print(dataset[0]["text"])
print(dataset.shape)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, eve

In [ ]:

text_data = dataset["text"]

print(len(text_data))

25000


In [ ]:

tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text", "label"])

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [ ]:


model = RobertaForMaskedLM.from_pretrained("roberta-base")

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

training_args = TrainingArguments(
    output_dir="./roberta-imdb-mlm",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
    logging_steps=100,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)



model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shahmeeramir (shahmeeramir-nuces) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
100,1.869600
200,1.932400
300,1.977800
400,1.918200
500,1.892400
600,1.957200
700,1.876600
800,1.934300
900,1.931200
1000,1.876700


TrainOutput(global_step=9375, training_loss=1.7845731372070313, metrics={'train_runtime': 2804.303, 'train_samples_per_second': 26.745, 'train_steps_per_second': 3.343, 'total_flos': 4936227436800000.0, 'train_loss': 1.7845731372070313, 'epoch': 3.0})

In [11]:
import torch

def predict_masked_text(text, model, tokenizer, mask_token):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    inputs = tokenizer(text, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    predicted_ids = torch.argmax(outputs.logits, dim=-1)
    predicted_text = tokenizer.decode(predicted_ids[0], skip_special_tokens=True)
    return predicted_text

In [15]:
# Example sentence
text = "The movie was <mask> and boring."  # For RoBERTa
bert_text = "The movie was [MASK] and boring."  # For BERT
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load BERT (if not already loaded)
bert_model = BertForMaskedLM.from_pretrained("bert-base-uncased").to(device)
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Get predictions
roberta_pred = predict_masked_text(text, model, tokenizer, "<mask>")
bert_pred = predict_masked_text(bert_text, bert_model, bert_tokenizer, "[MASK]")

print("Original:", text)
print("RoBERTa:", roberta_pred)
print("BERT:", bert_pred)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Original: The movie was <mask> and boring.
RoBERTa: .The movie was boring and boring.
BERT: . the movie was boring and boring..


In [14]:
from transformers import pipeline
mlm_fill = pipeline("fill-mask", model=model, tokenizer=tokenizer)
mlm_fill("The movie was <mask>.")

Device set to use cuda:0


[{'score': 0.14203035831451416,
  'token': 205,
  'token_str': ' good',
  'sequence': 'The movie was good.'},
 {'score': 0.09747723489999771,
  'token': 372,
  'token_str': ' great',
  'sequence': 'The movie was great.'},
 {'score': 0.06262505799531937,
  'token': 4206,
  'token_str': ' excellent',
  'sequence': 'The movie was excellent.'},
 {'score': 0.04194355383515358,
  'token': 4954,
  'token_str': ' OK',
  'sequence': 'The movie was OK.'},
 {'score': 0.030940866097807884,
  'token': 2679,
  'token_str': ' interesting',
  'sequence': 'The movie was interesting.'}]